# IMPORTING LIBRARIES

In [1]:
from flask import Flask, render_template, Response, request
import cv2
import numpy as np
from flask_ngrok import run_with_ngrok
import base64
from PIL import Image
import io

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as F
from torchvision.transforms import Compose, ToTensor, Resize, Normalize

# FLASK

In [2]:
model = torch.load('best_model.pth')

In [26]:
app = Flask(__name__)
run_with_ngrok(app)

def predict(image):
    image = Image.fromarray(np.array(image)[...,:3])
    image = Compose([Resize((224, 224)), ToTensor()])(image)
    image = Compose([Normalize(
        mean = [0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225])]
                   )(image)
    model.eval()
    output = model(image.view(1, 3, 224, 224).to("cuda")).cpu().detach().squeeze().numpy()
    output = output>0.5
    output = output.astype('float32')
    output = cv2.cvtColor(output, cv2.COLOR_GRAY2BGR)
    output = Image.fromarray((output*255).astype(np.uint8))
    return output
   
    
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/test', methods=['GET','POST'])
def test():
    if request.method == "POST":
        x = request.data.decode("utf-8")
        filename = 'some_image.jpg'
        with open(filename, 'wb') as f:
            f.write(base64.b64decode(x))
        image = Image.open(filename)
        im = predict(image)
        im = im.transpose(Image.FLIP_LEFT_RIGHT)
        data = io.BytesIO()
        im.save(data, "JPEG")
        encoded_img_data = base64.b64encode(data.getvalue())
        return Response(encoded_img_data.decode('utf-8'), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Sep/2021 15:42:49] "POST /test HTTP/1.1" 200 -


 * Running on http://c222-103-226-144-209.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Sep/2021 15:42:55] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:42:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:00] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:00] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:04] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:08] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:17] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:30] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:36] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:41] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:52] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:43:58] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:44:02] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:44:07] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15:44:25] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2021 15: